In [3]:
import pandas as pd

df = pd.read_csv("raw-data/sales_5000000.csv")
df.head()

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,Australia and Oceania,Palau,Office Supplies,Online,H,2020-03-06,517073523,2020-03-26,2401,651.21,524.96,1563555.21,1260428.96,303126.25
1,Europe,Poland,Beverages,Online,L,2014-04-18,380507028,2014-05-26,9340,47.45,31.79,443183.00,296918.60,146264.40
2,North America,Canada,Cereal,Online,M,2019-01-08,504055583,2019-01-31,103,205.70,117.11,21187.10,12062.33,9124.77
3,Europe,Belarus,Snacks,Online,C,2018-01-19,954955518,2018-02-27,1414,152.58,97.44,215748.12,137780.16,77967.96
4,Middle East and North Africa,Oman,Cereal,Offline,H,2023-04-26,970755660,2023-06-02,7027,205.70,117.11,1445453.90,822931.97,622521.93


In [5]:
df["orderDateYear"] = df["Order Date"].apply(lambda x: int(x.split("-")[0]))

In [6]:
boolMap = df["orderDateYear"] < 2024
trainData = df[boolMap].reset_index(drop=True)

In [7]:
trainData = trainData.drop(columns=["orderDateYear", ])
trainData.to_csv("data/trainData.csv", index=False)

In [9]:
streamData = df[~boolMap].reset_index(drop=True)
streamData.head()

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit,orderDateYear
0,North America,Greenland,Personal Care,Online,C,2024-05-25,659343469,2024-06-14,3239,81.73,56.67,264723.47,183554.13,81169.34,2024
1,Central America and the Caribbean,Saint Lucia,Personal Care,Online,M,2024-01-22,731556689,2024-02-26,2736,81.73,56.67,223613.28,155049.12,68564.16,2024
2,Sub-Saharan Africa,Guinea,Cosmetics,Offline,L,2024-04-29,891908156,2024-05-19,4299,437.20,263.33,1879522.80,1132055.67,747467.13,2024
3,Europe,Sweden,Beverages,Offline,H,2024-07-22,653010880,2024-08-06,9286,47.45,31.79,440620.70,295201.94,145418.76,2024
4,Europe,Georgia,Cereal,Offline,H,2024-03-21,989041030,2024-05-03,8074,205.70,117.11,1660821.80,945546.14,715275.66,2024


In [10]:
streamData["orderDateMonth"] = streamData["Order Date"].apply(lambda x: int(x.split("-")[1]))
streamData.head()

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit,orderDateYear,orderDateMonth
0,North America,Greenland,Personal Care,Online,C,2024-05-25,659343469,2024-06-14,3239,81.73,56.67,264723.47,183554.13,81169.34,2024,5
1,Central America and the Caribbean,Saint Lucia,Personal Care,Online,M,2024-01-22,731556689,2024-02-26,2736,81.73,56.67,223613.28,155049.12,68564.16,2024,1
2,Sub-Saharan Africa,Guinea,Cosmetics,Offline,L,2024-04-29,891908156,2024-05-19,4299,437.20,263.33,1879522.80,1132055.67,747467.13,2024,4
3,Europe,Sweden,Beverages,Offline,H,2024-07-22,653010880,2024-08-06,9286,47.45,31.79,440620.70,295201.94,145418.76,2024,7
4,Europe,Georgia,Cereal,Offline,H,2024-03-21,989041030,2024-05-03,8074,205.70,117.11,1660821.80,945546.14,715275.66,2024,3


In [12]:
months = streamData["orderDateMonth"].unique()

for month in months:
    monthBoolMap = (streamData["orderDateMonth"] == month)
    monthStreamData = streamData[monthBoolMap].reset_index(drop=True)
    monthStreamData.drop(columns=["orderDateYear", "orderDateMonth"], inplace=True)
    monthStreamData.to_csv(f"data/streamData-{month}.csv", index=False)

In [13]:
!ls data

streamData-1.csv  streamData-4.csv  streamData-7.csv  trainData.csv
streamData-2.csv  streamData-5.csv  streamData-8.csv
streamData-3.csv  streamData-6.csv  streamData-9.csv


In [18]:
df.head(1)

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,Australia and Oceania,Palau,Office Supplies,Online,H,2020-03-06,517073523,2020-03-26,2401,651.21,524.96,1563555.21,1260428.96,303126.25


In [22]:
columns = ["Region", "Country", "Item Type", "Sales Channel", "Order Priority"]
spark_columns = ["region", "country", "item_type", "sales_channel", "order_priority"]

for column, spark_column in zip(columns, spark_columns):
    values = list(df[column].unique())
    with open(f"validation-values/{spark_column}", "w") as f:
        f.write(", ".join(values))

In [25]:
!ls validation-values/

country  item_type  order_priority  region  sales_channel
